# Download the roads data

### Request the ohsome API

Question: What length does the roads mapped with RapID (Microsoft buildings) in in every continent?

#### Import libs

In [1]:
import json
import os
import requests

### Get the data for every specified region

#### Define the url

In [2]:
base_url = "https://api.ohsome.org/v1"
endpoint = "/elements/length/groupBy/boundary/groupBy/tag"
url = base_url + endpoint

#### Define the function to generate the data

In [3]:
def data_gen(name):

    with open(f"{geojson_dir}{name}", "r") as file:
        bpolys = json.load(file)


    parameters = {
        "bpolys": json.dumps(bpolys),  # pass GeoJSON as string.
        "filter": "type: way and (highway in (motorway,\
                                              motorway_link, trunk, trunk_link,\
                                              primary, primary_link, secondary,\
                                              secondary_link, tertiary,\
                                              tertiary_link, unclassified,\
                                              residential, living_street, pedestrian, footway, cycleway, track, path)\
                                  or (highway=service and service=alley))",
        "groupByKey": "source",
        "groupByValues": "maxar",
        "format": "json",
        "time": "2018-01-01/2023-01-01/P1Y",
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    for value in parameters.values():
        assert value != "", "Please provide values for the parameters"

    response = requests.post(url, data=parameters, headers=headers)
    response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

    print("Response:")
    print(json.dumps(response.json(), indent=4))  # Pretty print response

    result = response.json()["groupByResult"]
    return result


#### Get the names of regions/geojson files

In [4]:
os.chdir('..')
home_dir = os.getcwd()
geojson_dir = os.path.join(home_dir, "geojson-continent-borders", "")

geojson_names = []
for (dir_path, dir_names, file_names) in os.walk(geojson_dir):  # give the path
    geojson_names.extend(file_names)
print(geojson_names)
print(os.getcwd())


['0_Africa.geojson', '1_Asia.geojson', '2_Australia.geojson', '3_North America.geojson', '4_Oceania.geojson', '5_South America.geojson', '6_Antarctica.geojson', '7_Europe.geojson']
c:\Users\milan\OneDrive - MUNI\VŠ\PhD\Zahraniční stáž\Work\HeiGIT_notebooks\analysis\ai-assisted-osm-mapping-stats


#### Get the required data by the geojson files (not overhelm the server)

In [5]:
os.chdir(geojson_dir) # change the working directory because of geojson files

data = []

for geojson_name in geojson_names:

    data.append(data_gen(geojson_name))

os.chdir(home_dir)  # change the working directory back to the home directory


Response:
{
    "attribution": {
        "url": "https://ohsome.org/copyrights",
        "text": "\u00a9 OpenStreetMap contributors"
    },
    "apiVersion": "1.9.0",
    "groupByResult": [
        {
            "groupByObject": [
                0,
                "remainder"
            ],
            "result": [
                {
                    "timestamp": "2018-01-01T00:00:00Z",
                    "value": 5454453763.93
                },
                {
                    "timestamp": "2019-01-01T00:00:00Z",
                    "value": 6470871387.27
                },
                {
                    "timestamp": "2020-01-01T00:00:00Z",
                    "value": 7902787575.05
                },
                {
                    "timestamp": "2021-01-01T00:00:00Z",
                    "value": 8821001791.09
                },
                {
                    "timestamp": "2022-01-01T00:00:00Z",
                    "value": 9619285252.63
                }

In [6]:
print(os.getcwd())


c:\Users\milan\OneDrive - MUNI\VŠ\PhD\Zahraniční stáž\Work\HeiGIT_notebooks\analysis\ai-assisted-osm-mapping-stats


In [7]:
downloaded_data = r"downloaded-data"
with open(f"{downloaded_data}\\roads_continents_data_18-23_ai.json", "w") as file:
    json.dump(data, file, indent=4)